In [3]:
pip install requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
pip install selenium webdriver-manager

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
pip install beautifulsoup4


[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



                                              0.0/147.9 kB ? eta -:--:--
     -------------------------------------- 147.9/147.9 kB 4.4 MB/s eta 0:00:00


In [2]:
pip install tabulate

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tabulate import tabulate

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
import json  # นำเข้าโมดูล JSON

# URL ของหน้าเว็บที่ต้องการดึงข้อมูล
url = 'https://notebookspec.com/pc/ranking'

# ตั้งค่า Chrome WebDriver โดยใช้ ChromeDriverManager
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# เปิด URL ที่ระบุในเบราว์เซอร์
driver.get(url)

# รอให้หน้าเว็บโหลดเสร็จสมบูรณ์
time.sleep(150)  # รอในตอนแรกเพื่อให้หน้าเว็บโหลด

href_list = []

# ค้นหาทุก div ที่มีคลาส "table-row" หลังจากการคลิก
table_rows = driver.find_elements(By.CLASS_NAME, "table-row")

# วนลูปผ่านรายการเพื่อค้นหา "title-col" และดึง "href" จาก "name"
for row in table_rows:
    try:
        # ค้นหา element ที่มีคลาส "title-col"
        title_col = row.find_element(By.CLASS_NAME, "title-col")
        
        # ภายใน "title-col" ค้นหา element "a" ที่มีคลาส "name" และดึง "href"
        name_element = title_col.find_element(By.CLASS_NAME, "name").find_element(By.TAG_NAME, "a")
        href = name_element.get_attribute("href")
        
        # เพิ่ม href ที่ดึงได้ลงในลิสต์
        href_list.append(href)
    except Exception as e:
        print("ไม่พบ element 'name' ใน 'title-col':", e)

# ปิดเบราว์เซอร์
driver.quit()

# บันทึก href_list ลงในไฟล์ JSON
with open('hrefs.json', 'w', encoding='utf-8') as f:
    json.dump(href_list, f, ensure_ascii=False, indent=4)

print("บันทึกข้อมูลลงในไฟล์ hrefs.json สำเร็จ")

ไม่พบ element 'name' ใน 'title-col': Message: no such element: Unable to locate element: {"method":"css selector","selector":".name"}
  (Session info: chrome=129.0.6668.60); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00976AB3+25587]
	(No symbol) [0x00909C54]
	(No symbol) [0x00802113]
	(No symbol) [0x00846F62]
	(No symbol) [0x008471AB]
	(No symbol) [0x0083D121]
	(No symbol) [0x0086ABE4]
	(No symbol) [0x0083D044]
	(No symbol) [0x0086AE34]
	(No symbol) [0x00885370]
	(No symbol) [0x0086A936]
	(No symbol) [0x0083BA73]
	(No symbol) [0x0083C4CD]
	GetHandleVerifier [0x00C54C63+3032483]
	GetHandleVerifier [0x00CA6B99+3368153]
	GetHandleVerifier [0x00A08F62+624802]
	GetHandleVerifier [0x00A107DC+655644]
	(No symbol) [0x0091260D]
	(No symbol) [0x0090F6D8]
	(No symbol) [0x0090F875]
	(No symbol) [0x00901CA6]
	BaseThreadInitThunk [0x761BFCC9+25]
	RtlGetAppContainerN

In [2]:
with open('hrefs.json', 'r', encoding='utf-8') as f:
    href_list = json.load(f)
len(href_list)

1000

In [4]:
with open('hrefs.json', 'r', encoding='utf-8') as f:
    href_list = json.load(f)

# ตั้งค่า WebDriver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# ลิสต์เพื่อเก็บข้อมูลที่ดึงมา
all_items = []

# ฟังก์ชันสำหรับดึงข้อมูลจากลิงก์
def extract_info_from_link(url):
    driver.get(url)
    time.sleep(1)  # รอให้เพจโหลดเสร็จ

    items = []
    
    try:
        # ค้นหา <ul> ที่มีคลาส "my-list-item"
        ul_element = driver.find_element(By.CLASS_NAME, "my-list-item")
        
        # ค้นหาทุก <li> ภายใน <ul>
        li_elements = ul_element.find_elements(By.CLASS_NAME, "spec-item")
        
        for li in li_elements:
            type_div = li.find_element(By.CLASS_NAME, "type").text.strip().replace(':', '')
            name_div = li.find_element(By.CLASS_NAME, "name")
            name_a = name_div.find_element(By.TAG_NAME, "a")
            item_name = name_a.text.strip()
            item_href = name_a.get_attribute("href")
            
            # เพิ่มข้อมูลในลิสต์
            items.append({
                type_div: item_name,
                "href": item_href
            })
    except Exception as e:
        print(f"Error extracting data from {url}: {e}")
    
    return items

# ดึงข้อมูลจากทุกลิงก์ใน href_list
for href in href_list:
    print(f"Processing URL: {href}")
    items = extract_info_from_link(href)
    all_items.append(items)

# ปิดเบราว์เซอร์
driver.quit()

# บันทึกข้อมูลที่จัดรูปแบบลงในไฟล์ JSON
with open('formatted_items.json', 'w', encoding='utf-8') as f:
    json.dump(all_items, f, ensure_ascii=False, indent=4)

print("Data has been saved to formatted_items.json")

Processing URL: https://notebookspec.com/pc/9682653
Processing URL: https://notebookspec.com/pc/9683091
Processing URL: https://notebookspec.com/pc/9676034
Processing URL: https://notebookspec.com/pc/9680924
Processing URL: https://notebookspec.com/pc/9681912
Processing URL: https://notebookspec.com/pc/9681939
Processing URL: https://notebookspec.com/pc/9682033
Processing URL: https://notebookspec.com/pc/9682342
Processing URL: https://notebookspec.com/pc/9682562
Processing URL: https://notebookspec.com/pc/9682579
Processing URL: https://notebookspec.com/pc/9683734
Processing URL: https://notebookspec.com/pc/9683978
Processing URL: https://notebookspec.com/pc/9684262
Processing URL: https://notebookspec.com/pc/9684681
Processing URL: https://notebookspec.com/pc/9675974
Processing URL: https://notebookspec.com/pc/9675975
Processing URL: https://notebookspec.com/pc/9675976
Processing URL: https://notebookspec.com/pc/9675977
Processing URL: https://notebookspec.com/pc/9675978
Processing U

In [5]:
with open('formatted_items.json', 'r', encoding='utf-8') as f:
    formatted_items = json.load(f)
len(formatted_items)

1000

In [6]:
formatted_items

[[{'CPU ': 'Intel Core i5-14500',
   'href': 'https://notebookspec.com/pc-cpu/515-intel-core-i5-14500.html'},
  {'MB ': 'ASUS ROG Strix B760-G Gaming WIFI',
   'href': 'https://notebookspec.com/pc-mb/1959-asus-rog-strix-b760-g-gaming-wifi.html'},
  {'VGA ': 'GALAX GeForce RTX 4060 Ti (1-Click OC)',
   'href': 'https://notebookspec.com/pc-vga/2006-galax-geforce-rtx-4060-ti-1-click-oc.html'},
  {'RAM ': 'Klevv Cras V RGB DDR5 48GB (24GBx2) 6000 White',
   'href': 'https://notebookspec.com/pc-ram/1594-klevv-cras-v-rgb-ddr5-48gb-24gbx2-6000-white.html'},
  {'SSD ': 'Lexar NM610 Pro 1TB',
   'href': 'https://notebookspec.com/pc-ssd/899-lexar-nm610-pro-1tb.html'},
  {'HDD ': 'Western Digital Blue 2TB WD20EZAZ',
   'href': 'https://notebookspec.com/pc-hdd/243-western-digital-blue-2tb-wd20ezaz.html'},
  {'PSU ': 'ITONAS Aurora ARGB 500W',
   'href': 'https://notebookspec.com/pc-psu/600-itonas-aurora-argb-500w.html'},
  {'CASE ': 'Xigmatek Lux S Black',
   'href': 'https://notebookspec.com/pc-c

In [9]:
import pandas as pd

def extract_href(component_list):
    return {list(comp.keys())[0].strip(): comp['href'] for comp in component_list}

# สร้าง DataFrame โดยดึงเฉพาะ href
df = pd.DataFrame([extract_href(item) for item in formatted_items])
print(df)

                                                   CPU  \
0    https://notebookspec.com/pc-cpu/515-intel-core...   
1    https://notebookspec.com/pc-cpu/527-amd-ryzen-...   
2    https://notebookspec.com/pc-cpu/505-amd-ryzen-...   
3    https://notebookspec.com/pc-cpu/520-intel-core...   
4    https://notebookspec.com/pc-cpu/467-amd-ryzen-...   
..                                                 ...   
995  https://notebookspec.com/pc-cpu/470-intel-core...   
996  https://notebookspec.com/pc-cpu/522-amd-ryzen-...   
997  https://notebookspec.com/pc-cpu/520-intel-core...   
998  https://notebookspec.com/pc-cpu/516-intel-core...   
999  https://notebookspec.com/pc-cpu/520-intel-core...   

                                                    MB  \
0    https://notebookspec.com/pc-mb/1959-asus-rog-s...   
1    https://notebookspec.com/pc-mb/1964-asus-tuf-g...   
2    https://notebookspec.com/pc-mb/1948-asrock-a62...   
3    https://notebookspec.com/pc-mb/1984-asus-rog-m...   
4    https://

In [12]:
import requests
from bs4 import BeautifulSoup

def scrape_specifications(url):
    if url is None:
        return None
    response = requests.get(url)
    data = {}

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        # หา element ที่มี class 'price' เพื่อดึงข้อมูลราคา
        price_div = soup.find('div', class_='price')

        # หา element ที่มี class 'specificate-list' เพื่อดึงข้อมูลสเปค
        divs_specificate_list = soup.find_all('div', class_='specificate-list')
        for div in divs_specificate_list:
            title = div.find('div', class_='specificate-title').get_text(strip=True)
            info = div.find('div', class_='specificate-info').get_text(strip=True)
            data[title] = info
        if price_div:
            price = price_div.get_text(strip=True)
            data['Price'] = price
    else:
        print(f"การเชื่อมต่อกับเว็บไซต์ล้มเหลว: Status code {response.status_code}")

    return data

In [19]:
all_dataCPU = []
for url in df['CPU']:
    if pd.notna(url):  # ตรวจสอบว่า url ไม่เป็น NaN
        scraped_data = scrape_specifications(url)
        all_dataCPU.append(scraped_data)
    else:
        all_dataCPU.append({'Brand': None, 'Series': None, 'Model': None, 'Base Clock': None, 'Price': None})  # กรณีที่ไม่มีข้อมูล

dfCPU = pd.DataFrame(all_dataCPU)

columns_CPU = ['Brand', 'Series', 'Model', 'Base Clock', 'Price']
for col in columns_CPU:
    if col not in dfCPU.columns:
        dfCPU[col] = None  # เพิ่มคอลัมน์ที่ขาดหาย

# แสดงผลลัพธ์
print(tabulate(dfCPU[columns_CPU], headers='keys', tablefmt='grid'))

+-----+---------+--------------------+---------------------------------+--------------+---------+
|     | Brand   | Series             | Model                           | Base Clock   | Price   |
+=====+=========+====================+=================================+==============+=========+
|   0 | Intel   | Core i5            | Core i5-14500                   | 2.60 GHz     | 8,960   |
+-----+---------+--------------------+---------------------------------+--------------+---------+
|   1 | AMD     | Ryzen 5            | Ryzen 5 7500F                   | 3.70 GHz     | 6,810   |
+-----+---------+--------------------+---------------------------------+--------------+---------+
|   2 | AMD     | Ryzen 5            | Ryzen 5 8500G                   | 3.50 GHz     | 6,840   |
+-----+---------+--------------------+---------------------------------+--------------+---------+
|   3 | Intel   | Core i9            | Core i9-14900KS Special Edition | 3.20 GHz     | 27,770  |
+-----+---------+---

In [23]:
dfCPU.to_csv('spec\dfCPU_full.csv', index=False)
dfCPU[columns_CPU].to_csv('spec\dfCPU.csv', index=False)

In [24]:
all_dataMainboard = []

for url in df['MB']:
    if pd.notna(url):  # ตรวจสอบว่า url ไม่เป็น NaN
        scraped_data = scrape_specifications(url)
        all_dataMainboard.append(scraped_data)
    else:
        all_dataMainboard.append({'Brand': None, 'Model': None, 'CPU Support': None, 'Memory Max.': None, 'Memory Support': None, 'Price': None})  # กรณีที่ไม่มีข้อมูล

dfMainboard = pd.DataFrame(all_dataMainboard)

columns_Mainboard = ['Brand', 'Model', 'CPU Support', 'Memory Max.', 'Memory Support', 'Price']
for col in columns_Mainboard:
    if col not in dfMainboard.columns:
        dfMainboard[col] = None  # เพิ่มคอลัมน์ที่ขาดหาย

# แสดงผลลัพธ์
print(tabulate(dfMainboard[columns_Mainboard], headers='keys', tablefmt='grid'))

+-----+----------+-------------------------------+----------------------------------------------------------------------------------------------------+---------------+-------------------+-----------+
|     | Brand    | Model                         | CPU Support                                                                                        | Memory Max.   | Memory Support    | Price     |
+=====+==========+===============================+====================================================================================================+===============+===================+===========+
|   0 | ASUS     | ROG Strix B760-G Gaming WIFI  | Intel 14th Generation / Intel 13th Generation / Intel 12th Generation / Pentium / Celeron          | 192GB         | Dual Channel DDR5 | 7,490     |
+-----+----------+-------------------------------+----------------------------------------------------------------------------------------------------+---------------+-------------------+-----------+


In [25]:
dfMainboard.to_csv('spec\dfMainboard_full.csv', index=False)
dfMainboard[columns_Mainboard].to_csv('spec\dfMainboard.csv', index=False)

In [26]:
all_dataGraphicCard = []

for url in df['VGA']:
    if pd.notna(url):  # ตรวจสอบว่า url ไม่เป็น NaN
        scraped_data = scrape_specifications(url)
        all_dataGraphicCard.append(scraped_data)
    else:
        all_dataGraphicCard.append({'Brand': None, 'GPU Chipset': None, 'Series': None, 'Model': None, 'GPU Clock': None, 'Boost Clock': None, 'Memory Clock': None, 'Memory Size': None, 'Price': None})  # กรณีที่ไม่มีข้อมูล

dfGraphicCard = pd.DataFrame(all_dataGraphicCard)

columns_GraphicCard = ['Brand', 'GPU Chipset', 'Series', 'Model', 'GPU Clock', 'Boost Clock', 'Memory Clock', 'Memory Size', 'Price']
for col in columns_GraphicCard:
    if col not in dfGraphicCard.columns:
        dfGraphicCard[col] = None  # เพิ่มคอลัมน์ที่ขาดหาย

# แสดงผลลัพธ์
print(tabulate(dfGraphicCard[columns_GraphicCard], headers='keys', tablefmt='grid'))

+-----+-------------+---------------+--------------------------+-----------------------------------------------------+-------------+------------------+----------------+---------------+-----------+
|     | Brand       | GPU Chipset   | Series                   | Model                                               | GPU Clock   | Boost Clock      | Memory Clock   | Memory Size   | Price     |
+=====+=============+===============+==========================+=====================================================+=============+==================+================+===============+===========+
|   0 | GALAX       | NVIDIA        | GeForce RTX 40 Series    | GeForce RTX 4060 Ti (1-Click OC)                    | 2310 MHz    | 2565 MHz         | 18000 MHz      | 8 GB          | 13,700    |
+-----+-------------+---------------+--------------------------+-----------------------------------------------------+-------------+------------------+----------------+---------------+-----------+
|   1 | GIGABYT

In [27]:
dfGraphicCard.to_csv('spec\dfGraphicCard_full.csv', index=False)
dfGraphicCard[columns_GraphicCard].to_csv('spec\dfGraphicCard.csv', index=False)

In [28]:
all_dataRAM = []

for url in df['RAM']:
    if pd.notna(url):  # ตรวจสอบว่า url ไม่เป็น NaN
        scraped_data = scrape_specifications(url)
        all_dataRAM.append(scraped_data)
    else:
        all_dataRAM.append({'Memory Size': None,'Memory Speed': None, 'Price': None})  # กรณีที่ไม่มีข้อมูล

dfRAM = pd.DataFrame(all_dataRAM)

columns_RAM = ['Memory Size','Memory Speed', 'Price']
for col in columns_RAM:
    if col not in dfRAM.columns:
        dfRAM[col] = None  # เพิ่มคอลัมน์ที่ขาดหาย

# แสดงผลลัพธ์
print(tabulate(dfRAM[columns_RAM], headers='keys', tablefmt='grid'))

+-----+------------------+----------------+----------+
|     | Memory Size      | Memory Speed   | Price    |
+=====+==================+================+==========+
|   0 | 48GB (24GBx2)    | 6000 MHz       | 3,090 .- |
+-----+------------------+----------------+----------+
|   1 | 16GB             | 5200 MHz       | 1,382    |
+-----+------------------+----------------+----------+
|   2 | 16GB (8GBx2)     | 5200 MHz       | 2,080    |
+-----+------------------+----------------+----------+
|   3 | 32GB (16GBx2)    | 6400 MHz       | 5,630    |
+-----+------------------+----------------+----------+
|   4 | 16GB (8GBx2)     | 3600 MHz       | 1,442    |
+-----+------------------+----------------+----------+
|   5 | 16GB (8GBx2)     | 5200 MHz       | 2,080    |
+-----+------------------+----------------+----------+
|   6 | 16GB (8GBx2)     | 4800 MHz       | 2,177    |
+-----+------------------+----------------+----------+
|   7 | 32GB (16GBx2)    | 6000 MHz       | 4,840 .- |
+-----+---

In [29]:
dfRAM.to_csv('spec\dfRAM_full.csv', index=False)
dfRAM[columns_RAM].to_csv('spec\dfRAM.csv', index=False)

In [30]:
all_dataSSD = []

for url in df['SSD']:
    if pd.notna(url):  # ตรวจสอบว่า url ไม่เป็น NaN
        scraped_data = scrape_specifications(url)
        all_dataSSD.append(scraped_data)
    else:
        all_dataSSD.append({'Capacity': None, 'Read / Write': None, 'Price': None})  # กรณีที่ไม่มีข้อมูล

dfSSD = pd.DataFrame(all_dataSSD)

columns_SSD = ['Capacity', 'Read / Write', 'Price']
for col in columns_SSD:
    if col not in dfSSD.columns:
        dfSSD[col] = None  # เพิ่มคอลัมน์ที่ขาดหาย

# แสดงผลลัพธ์
print(tabulate(dfSSD[columns_SSD], headers='keys', tablefmt='grid'))

+-----+------------+----------------------------------------------------------------+-----------+
|     | Capacity   | Read / Write                                                   | Price     |
+=====+============+================================================================+===========+
|   0 | 1TB        | Up to 3300 /                                                   | 785 .-    |
|     |            |                                                     2600 MB/s  |           |
+-----+------------+----------------------------------------------------------------+-----------+
|   1 | 2TB        | Up to 3500 /                                                   | 4,350     |
|     |            |                                                     2800 MB/s  |           |
+-----+------------+----------------------------------------------------------------+-----------+
|   2 | 512GB      | Up to 3600 /                                                   | 1,490     |
|     |            |

In [31]:
dfSSD.to_csv('spec\dfSSD_full.csv', index=False)
dfSSD[columns_SSD].to_csv('spec\dfSSD.csv', index=False)

In [32]:
all_dataSSD2 = []
for url in df['SSD2']:
    if pd.notna(url):  # ตรวจสอบว่า url ไม่เป็น NaN
        scraped_data = scrape_specifications(url)
        all_dataSSD2.append(scraped_data)
    else:
        all_dataSSD2.append({'Capacity': None, 'Read / Write': None, 'Price': None})  # กรณีที่ไม่มีข้อมูล

dfSSD2 = pd.DataFrame(all_dataSSD2)

columns_SSD2 = ['Capacity', 'Read / Write', 'Price']
for col in columns_SSD2:
    if col not in dfSSD2.columns:
        dfSSD2[col] = None  # เพิ่มคอลัมน์ที่ขาดหาย

# แสดงผลลัพธ์
print(tabulate(dfSSD2[columns_SSD2], headers='keys', tablefmt='grid'))

+-----+------------+---------------------------------------------------------------+-----------+
|     | Capacity   | Read / Write                                                  | Price     |
+=====+============+===============================================================+===========+
|   0 |            |                                                               |           |
+-----+------------+---------------------------------------------------------------+-----------+
|   1 |            |                                                               |           |
+-----+------------+---------------------------------------------------------------+-----------+
|   2 |            |                                                               |           |
+-----+------------+---------------------------------------------------------------+-----------+
|   3 | 1TB        | Up to 550 /                                                   | 3,450     |
|     |            |          

In [44]:
dfSSD2.to_csv('spec\dfSSD2_full.csv', index=False)
dfSSD2[columns_SSD2].to_csv('spec\dfSSD2.csv', index=False)

In [34]:
all_dataHDD = []

for url in df['HDD']:
    if pd.notna(url):  # ตรวจสอบว่า url ไม่เป็น NaN
        scraped_data = scrape_specifications(url)
        all_dataHDD.append(scraped_data)
    else:
        all_dataHDD.append({'Capacity': None, 'Rotational Speed': None, 'Price': None})  # กรณีที่ไม่มีข้อมูล

dfHDD = pd.DataFrame(all_dataHDD)

columns_HDD = ['Capacity', 'Rotational Speed', 'Price']
for col in columns_HDD:
    if col not in dfHDD.columns:
        dfHDD[col] = None  # เพิ่มคอลัมน์ที่ขาดหาย

# แสดงผลลัพธ์
print(tabulate(dfHDD[columns_HDD], headers='keys', tablefmt='grid'))

+-----+------------+--------------------+----------+
|     | Capacity   | Rotational Speed   | Price    |
+=====+============+====================+==========+
|   0 | 2TB        | 5400 RPM           | 1,850    |
+-----+------------+--------------------+----------+
|   1 |            |                    |          |
+-----+------------+--------------------+----------+
|   2 |            |                    |          |
+-----+------------+--------------------+----------+
|   3 | 4TB        | 7200 RPM           | 5,700    |
+-----+------------+--------------------+----------+
|   4 |            |                    |          |
+-----+------------+--------------------+----------+
|   5 |            |                    |          |
+-----+------------+--------------------+----------+
|   6 | 1TB        | 7200 RPM           | 1,825    |
+-----+------------+--------------------+----------+
|   7 |            |                    |          |
+-----+------------+--------------------+-----

In [35]:
dfHDD.to_csv('spec\dfHDD_full.csv', index=False)
dfHDD[columns_HDD].to_csv('spec\dfHDD.csv', index=False)

In [36]:
all_dataPS = []

for url in df['PSU']:
    if pd.notna(url):  # ตรวจสอบว่า url ไม่เป็น NaN
        scraped_data = scrape_specifications(url)
        all_dataPS.append(scraped_data)
    else:
        all_dataPS.append({'Maximum Power': None, 'Price': None})  # กรณีที่ไม่มีข้อมูล

dfPS = pd.DataFrame(all_dataPS)

columns_PS = ['Maximum Power', 'Price']
for col in columns_PS:
    if col not in dfPS.columns:
        dfPS[col] = None  # เพิ่มคอลัมน์ที่ขาดหาย

# แสดงผลลัพธ์
print(tabulate(dfPS[columns_PS], headers='keys', tablefmt='grid'))

+-----+-----------------+-----------+
|     | Maximum Power   | Price     |
+=====+=================+===========+
|   0 | 500W            | 1,250     |
+-----+-----------------+-----------+
|   1 | 850W            | 1,890     |
+-----+-----------------+-----------+
|   2 | 550W            | 1,275     |
+-----+-----------------+-----------+
|   3 | 1650W           | 10,170    |
+-----+-----------------+-----------+
|   4 | 600W            | 890       |
+-----+-----------------+-----------+
|   5 | 600W            | 1,525     |
+-----+-----------------+-----------+
|   6 | 600W            | 1,525     |
+-----+-----------------+-----------+
|   7 | 750W            | 5,900     |
+-----+-----------------+-----------+
|   8 | 500W            | 1,250     |
+-----+-----------------+-----------+
|   9 | 500W            | 640       |
+-----+-----------------+-----------+
|  10 | 650W            | 1,690     |
+-----+-----------------+-----------+
|  11 | 1000W           | 4,990     |
+-----+-----

In [37]:
dfPS.to_csv('spec\dfPS_full.csv', index=False)
dfPS[columns_PS].to_csv('spec\dfPS.csv', index=False)

In [38]:
all_dataCASE = []

for url in df['CASE']:
    if pd.notna(url):  # ตรวจสอบว่า url ไม่เป็น NaN
        scraped_data = scrape_specifications(url)
        all_dataCASE.append(scraped_data)
    else:
        all_dataCASE.append({'Brand': None, 'Model': None, 'Weight': None, 'I/O Ports': None, 'Price': None})  # กรณีที่ไม่มีข้อมูล

dfCASE = pd.DataFrame(all_dataCASE)

columns_CASE = ['Brand', 'Model', 'Weight', 'I/O Ports', 'Price']
for col in columns_CASE:
    if col not in dfCASE.columns:
        dfCASE[col] = None  # เพิ่มคอลัมน์ที่ขาดหาย

# แสดงผลลัพธ์
print(tabulate(dfCASE[columns_CASE], headers='keys', tablefmt='grid'))

+-----+-------------------+----------------------------------------+------------+------------------------------------------------------------------------+-----------+
|     | Brand             | Model                                  | Weight     | I/O Ports                                                              | Price     |
+=====+===================+========================================+============+========================================================================+===========+
|   0 | Xigmatek          | Lux S Black                            | -Kg        | 2 x USB 2.0 / 1 x USB 3.0 / 1 x Audio Header                           | 1,990     |
+-----+-------------------+----------------------------------------+------------+------------------------------------------------------------------------+-----------+
|   1 | Montech           | X2 Mesh White                          | -Kg        | 2 x USB 2.0 / 1 x USB 3.0 / 1 x Audio Header                           | 1,300     

In [43]:
dfCASE.to_csv('spec\dfCASE_full.csv', index=False)
dfCASE[columns_CASE].to_csv('spec\dfCASE.csv', index=False)

In [39]:
all_dataCOOLING = []

for url in df['COOLING']:
    if pd.notna(url):  # ตรวจสอบว่า url ไม่เป็น NaN
        scraped_data = scrape_specifications(url)
        all_dataCOOLING.append(scraped_data)
    else:
        all_dataCOOLING.append({'Brand': None, 'Model': None, 'Fan Built-In': None, 'Price': None})  # กรณีที่ไม่มีข้อมูล

dfCOOLING = pd.DataFrame(all_dataCOOLING)

columns_COOLING = ['Brand', 'Model', 'Fan Built-In', 'Price']
for col in columns_COOLING:
    if col not in dfCOOLING.columns:
        dfCOOLING[col] = None  # เพิ่มคอลัมน์ที่ขาดหาย

# แสดงผลลัพธ์
print(tabulate(dfCOOLING[columns_COOLING], headers='keys', tablefmt='grid'))

+-----+---------------+---------------------------------------+----------------+---------+
|     | Brand         | Model                                 | Fan Built-In   | Price   |
+=====+===============+=======================================+================+=========+
|   0 |               |                                       |                |         |
+-----+---------------+---------------------------------------+----------------+---------+
|   1 | Cooler Master | Masterliquid 240L Core ARGB White     | 2 x 120mm      | 2,850   |
+-----+---------------+---------------------------------------+----------------+---------+
|   2 |               |                                       |                |         |
+-----+---------------+---------------------------------------+----------------+---------+
|   3 | ASUS          | ROG Ryujin III 360                    | 3 x 120mm      | 10,980  |
+-----+---------------+---------------------------------------+----------------+---------+

In [40]:
dfCOOLING.to_csv('spec\dfCOOLING_full.csv', index=False)
dfCOOLING[columns_COOLING].to_csv('spec\dfCOOLING.csv', index=False)

In [41]:
all_dataMONITOR = []

for url in df['MONITOR']:
    if pd.notna(url):  # ตรวจสอบว่า url ไม่เป็น NaN
        scraped_data = scrape_specifications(url)
        all_dataMONITOR.append(scraped_data)
    else:
        all_dataMONITOR.append({'Brand': None, 'Model': None, 'Display Size': None, 'Max Resolution': None, 'Refresh Rate': None, 'Price': None})  # กรณีที่ไม่มีข้อมูล

dfMONITOR = pd.DataFrame(all_dataMONITOR)

columns_MONITOR = ['Brand', 'Model', 'Display Size', 'Max Resolution', 'Refresh Rate', 'Price']
for col in columns_MONITOR:
    if col not in dfMONITOR.columns:
        dfMONITOR[col] = None  # เพิ่มคอลัมน์ที่ขาดหาย

# แสดงผลลัพธ์
print(tabulate(dfMONITOR[columns_MONITOR], headers='keys', tablefmt='grid'))

+-----+-----------+-------------------------------------------+----------------+------------------+----------------+-----------+
|     | Brand     | Model                                     | Display Size   | Max Resolution   | Refresh Rate   | Price     |
+=====+===========+===========================================+================+==================+================+===========+
|   0 | AOC       | 27V5BK/67                                 | 27"            | 1920 x 1080      | 75Hz           | 3,930     |
+-----+-----------+-------------------------------------------+----------------+------------------+----------------+-----------+
|   1 | DAHUA     | DHI-LM24-E231                             | 23.8"          | 1920 x 1080      | 165Hz          | 3,475     |
+-----+-----------+-------------------------------------------+----------------+------------------+----------------+-----------+
|   2 |           |                                           |                |                 

In [42]:
dfMONITOR.to_csv('spec\dfMONITOR_full.csv', index=False)
dfMONITOR[columns_MONITOR].to_csv('spec\dfMONITOR.csv', index=False)

In [45]:
CPU = pd.read_csv('spec\dfCPU.csv')
Mainboard = pd.read_csv('spec\dfMainboard.csv')
GraphicCard = pd.read_csv('spec\dfGraphicCard.csv')
RAM = pd.read_csv('spec\dfRAM.csv')
SSD = pd.read_csv('spec\dfSSD.csv')
SSD2 = pd.read_csv('spec\dfSSD2.csv')
HDD = pd.read_csv('spec\dfHDD.csv')
PS = pd.read_csv('spec\dfPS.csv')
CASE = pd.read_csv('spec\dfCASE.csv')
COOLING = pd.read_csv('spec\dfCOOLING.csv')
MONITOR = pd.read_csv('spec\dfMONITOR.csv')

SpecCom = pd.concat([CPU, Mainboard, GraphicCard, RAM, SSD, SSD2, HDD, PS, CASE, COOLING, MONITOR], axis=1)

SpecCom.to_csv('spec\SpecCom.csv', index=False)


In [46]:
Qspec = pd.read_csv('spec\SpecCom.csv')
Qspec

,Brand,Series,Model,Base Clock,Price,Brand.1,Model.1,CPU Support,Memory Max.,Memory Support,...,Brand.4,Model.4,Fan Built-In,Price.9,Brand.5,Model.5,Display Size,Max Resolution,Refresh Rate,Price.10
0,Intel,Core i5,Core i5-14500,2.60 GHz,"8,960",ASUS,ROG Strix B760-G Gaming WIFI,Intel 14th Generation / Intel 13th Generation ...,192GB,Dual Channel DDR5,...,NaN,NaN,NaN,NaN,AOC,27V5BK/67,"27""",1920 x 1080,75Hz,"3,930"
1,AMD,Ryzen 5,Ryzen 5 7500F,3.70 GHz,"6,810",ASUS,TUF Gaming A620M-Plus WIFI,AMD Socket AM5 Ryzen 8000 and 7000 Series Proc...,192GB,Dual Channel DDR5,...,Cooler Master,Masterliquid 240L Core ARGB White,2 x 120mm,"2,850",DAHUA,DHI-LM24-E231,"23.8""",1920 x 1080,165Hz,"3,475"
2,AMD,Ryzen 5,Ryzen 5 8500G,3.50 GHz,"6,840",ASRock,A620M-HDV/M.2,AMD Socket AM5 Ryzen 8000 and 7000 Series Proc...,96GB,Dual Channel DDR5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Intel,Core i9,Core i9-14900KS Special Edition,3.20 GHz,"27,770",ASUS,ROG Maximus Z790 Apex Encore,Intel 14th Generation / Intel 13th Generation ...,96GB,Dual Channel DDR5,...,ASUS,ROG Ryujin III 360,3 x 120mm,"10,980",DELL,Alienware AW3821DW,"37""",3840 x 1600,144Hz,"54,500"
4,AMD,Ryzen 5,Ryzen 5 4600G,3.70 GHz,"3,550",ASUS,B450M-A Pro Max,AMD Ryzen 1000 / 2000 / 3000 series / AMD Ryze...,64GB,Dual Channel DDR4,...,DeepCool,AG400,1 x 120mm,610,DAHUA,LM22-B200S,"21.5""",1920 x 1080,100Hz,"1,890"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Intel,Core i5,Core i5-12400F,2.50 GHz,"3,940",ASUS,Prime B760M-A WIFI,Intel 14th Generation / Intel 13th Generation ...,128GB,Dual Channel DDR5,...,Thermaltake,TH360 ARGB Sync,3 x 120mm,"3,290",NaN,NaN,NaN,NaN,NaN,NaN
996,AMD,Ryzen 7,Ryzen 7 8700F,4.10 GHz,"10,640",GIGABYTE,B650M D3HP,AMD Socket AM5 Ryzen 8000 and 7000 Series Proc...,192GB,Dual Channel DDR5,...,Thermaltake,TH240 V2 ARGB Sync,2 x 120mm,"2,590",ASUS,ProArt PA328QV,"31.5""",2560 x 1440,75Hz,"14,050"
997,Intel,Core i9,Core i9-14900KS Special Edition,3.20 GHz,"27,770",GIGABYTE,B760 AORUS Elite AX,Intel 14th Generation / Intel 13th Generation ...,192GB,Dual Channel DDR5,...,CORSAIR,iCUE H150i Elite LCD XT,3 x 120mm,"7,890",NaN,NaN,NaN,NaN,NaN,NaN
998,Intel,Core i7,Core i7-14700F,2.10 GHz,"13,420",MSI,Z790 Gaming Plus WIFI,Intel 14th Generation / Intel 13th Generation ...,256GB,Dual Channel DDR5,...,Cooler Master,Masterliquid 360L Core ARGB,3 x 120mm,"3,650",NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

In [5]:
url = 'https://ihavecpu.com/category/promotion?addition=504'

href_list = []

# ตั้งค่า Chrome WebDriver โดยใช้ ChromeDriverManager
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# เปิด URL ที่ระบุในเบราว์เซอร์
driver.get(url)

time.sleep(3)  # รอในตอนแรกเพื่อให้หน้าเว็บโหลด

# ค้นหาทุก div ที่มีคลาส "table-row"
table_rows = driver.find_elements(By.CLASS_NAME, "image-holder")

# ปริ้น HTML ของแต่ละ div
for row in table_rows:
    name_a = row.find_element(By.TAG_NAME, "a")
    item_href = name_a.get_attribute("href")
    href_list.append(item_href)

driver.quit()


In [6]:
table = []
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
for link in href_list:
    keyla = []
    driver.get(link)
    time.sleep(2)
    link_01 = driver.find_element(By.TAG_NAME, "tbody")
    link_02 = link_01.find_elements(By.TAG_NAME, "tr")
    for link_03 in link_02:
        title = link_03.find_element(By.CLASS_NAME, "column1").text
        row = link_03.find_element(By.CLASS_NAME, "column2").text
        keyla.append({
            title: row
        })
    table.append(keyla)
    
driver.quit()

for Prin in table:
    print(Prin)

[{'CPU': 'AMD Ryzen™ 5 4500 3.6GHz 6C/12T'}, {'Mainboard': 'GIGABYTE A520M K V2 (REV.1.1)'}, {'Graphic card': 'PALIT GEFORCE RTX 3050 STORM X 6GB GDDR6'}, {'Memory': 'KINGSTON FURY BEAST 16GB (8x2) DDR4 3200MHz BLACK'}, {'Storage': 'M.2 LEXAR x iHAVECPU NM620 256GB Read (3,500 MB/s)'}, {'Power Supply': 'AZZA PSAZ 550W 80+ BRONZE'}, {'Case': 'iHAVECPU IHC 03A (BLACK)(mATX) | (เลือกเคสติดต่อ ADMIN)'}, {'Warranty': '3 Years (ยกเว้น CASE ประกัน 1 ปี)'}]
[{'CPU': 'AMD Ryzen™ 5 5500 3.6GHz 6C/12T'}, {'Mainboard': 'GIGABYTE A520M K V2 (REV.1.1)'}, {'Graphic card': 'PALIT GEFORCE RTX 3050 STORM X 6GB GDDR6'}, {'Memory': 'KINGSTON FURY BEAST 16GB (8x2) DDR4 3200MHz BLACK'}, {'Storage': 'M.2 LEXAR x iHAVECPU NM620 256GB Read (3,500 MB/s)'}, {'Power Supply': 'AZZA PSAZ 550W 80+ BRONZE'}, {'Case': 'iHAVECPU IHC 03A (WHITE)(mATX) | (เลือกเคสติดต่อ ADMIN)'}, {'Warranty': '3 Years (ยกเว้น CASE ประกัน 1 ปี)'}]
[{'CPU': 'Intel® CORE I3-12100F 3.3GHz 4C/8T'}, {'Mainboard': 'ASROCK H610M-HVS/M.2 R2.0'}, 

In [8]:
import json
print(len(table))

with open('hrefs.json', 'w', encoding='utf-8') as f:
    json.dump(table, f, ensure_ascii=False, indent=4)


6


In [ ]:
url = 'https://www.overclockers.co.uk/pc-systems/gaming-pcs'

href_list = []

# ตั้งค่า Chrome WebDriver โดยใช้ ChromeDriverManager
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# เปิด URL ที่ระบุในเบราว์เซอร์
driver.get(url)

time.sleep(3)

table_rows = driver.find_elements(By.CLASS_NAME, "col")

for to in table_rows:
    print(to.text)




# ค้นหาทุก div ที่มีคลาส "table-row"


driver.quit()



AttributeError: 'WebElement' object has no attribute 'page_source'